In [ ]:
!sudo apt install python3.8-full python3-pip
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!pip install auto-sklearn

In [ ]:
import pandas as pd
import autosklearn
import pickle

In [ ]:
import autosklearn


features = features[features['id'].isin(train_ids['gma_id']) | features['id'].isin(val_ids['gma_id'])]
test_features = features[features['id'].isin(test_ids['gma_id'])]
holdout_features = features[features['id'].isin(holdout_ids['gma_id'])]


pivot_df = features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
risk_df = features[['infant', 'risk']].drop_duplicates()

test_pivot_df = test_features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
test_risk_df = test_features[['infant', 'risk']].drop_duplicates()

holdout_pivot_df = holdout_features.pivot_table(index='infant', columns=['feature'], values='Value', fill_value=0)
holdout_risk_df = holdout_features[['infant', 'risk']].drop_duplicates()

train_df = pd.merge(pivot_df, risk_df, on='infant', how='inner')
test_df = pd.merge(test_pivot_df, test_risk_df, on='infant', how='inner')
holdout_df = pd.merge(holdout_pivot_df, holdout_risk_df, on='infant', how='inner')

X = train_df.drop(columns=['infant', 'risk'])
y = train_df['risk']

X_test = test_df.drop(columns=['infant', 'risk'])
y_test = test_df['risk']

X_holdout = holdout_df.drop(columns=['infant', 'risk'])
y_holdout = holdout_df['risk']

In [ ]:
  automl = AutoSklearn2Classifier(
      ensemble_size=1,
      dataset_compression=False,
      allow_string_features=False,
      time_left_for_this_task=300,
      per_run_time_limit=30,
      metric=autosklearn.metrics.balanced_accuracy,
      delete_tmp_folder_after_terminate=False,
      memory_limit=None,
      disable_evaluator_output=False,
  )

automl.fit(X_train, y_train)
probabilities = automl.predict_proba(X_holdout)[:, 1]

In [ ]:
!wget --content-disposition --max-redirect=20 https://osf.io/download/xa43g/
model = pickle.load(open('automl_vanilla_gma_prediction.pkl', 'rb'))
